In [ ]:
import numpy as np
import pandas as pd
import cv2
from glob import glob
from PIL import Image
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import time
from tqdm import tqdm

from endecoder import *

In [ ]:
# train 데이터 총 279309
root = r"D:\노지 작물 질병 진단 이미지"
train_dir = os.path.join(root, "train")
label_dir = os.path.join(train_dir, "label")

# 질병데이터
label_files = glob(os.path.join(label_dir, "*_0"))
df0 = {label_files[i].split("\\")[-1]:len(glob(os.path.join(label_dir, label_files[i].split("\\")[-1]) + '/*')) for i in range(10)}

# 정상데이터
label_files = glob(os.path.join(label_dir, "*_1"))
df1 = {label_files[i].split("\\")[-1]:len(glob(os.path.join(label_dir, label_files[i].split("\\")[-1]) + '/*')) for i in range(10)}

# 증강데이터
label_files = glob(os.path.join(label_dir, "*_2"))
df2 = {label_files[i].split("\\")[-1]:len(glob(os.path.join(label_dir, label_files[i].split("\\")[-1]) + '/*')) for i in range(10)}

df = pd.DataFrame({"disease":list(df0.values()),
             "real" : list(df1.values()),
             "augmentation":list(df2.values()),}, index=list(df1.keys()) )

In [ ]:
# 검증 데이터 총 34921 개
vaild_dir = os.path.join(root, "valid")
label_dir = os.path.join(vaild_dir, "label")

# 질병데이터
label_files = glob(os.path.join(label_dir, "*_0"))
dfv0 = {label_files[i].split("\\")[-1]:len(glob(os.path.join(label_dir, label_files[i].split("\\")[-1]) + '/*')) for i in range(10)}

# 정상데이터
label_files = glob(os.path.join(label_dir, "*_1"))
dfv1 = {label_files[i].split("\\")[-1]:len(glob(os.path.join(label_dir, label_files[i].split("\\")[-1]) + '/*')) for i in range(10)}

# 증강데이터
label_files = glob(os.path.join(label_dir, "*_2"))
dfv2 = {label_files[i].split("\\")[-1]:len(glob(os.path.join(label_dir, label_files[i].split("\\")[-1]) + '/*')) for i in range(10)}

dfv = pd.DataFrame({"disease":list(dfv0.values()),
             "real" : list(dfv1.values()),
             "augmentation":list(dfv2.values()),}, index=list(dfv1.keys()) )

In [ ]:
279309 / (279309 + 34921)
# train : test = 0.8888680266047163 : 0.12222

In [ ]:
# 고추 바이러스 데이터들

root = r"D:\[라벨]02.고추_1.질병"
train_dir = os.path.join(root, "라벨링데이터")
# label_dir = os.path.join(train_dir, "label")
files = glob('D:\고추질병라벨\*')

In [ ]:
pepper1 = []
pepper2 = []
for path in files:
    data = json.load(open(path, "r"))
    if data['annotations']['disease'] == 3:
        pepper1.append(path)
    elif data['annotations']['disease'] == 4:
        pepper2.append(path)

In [ ]:
import shutil
root = r'D:\고추ood'
train_dir = os.path.join(root, "label")
label_dir1 = os.path.join(train_dir, "p1")
label_dir2 = os.path.join(train_dir, "p2")
img_root = r"D:\02.고추_1질병"
save_root1 = r"D:\고추ood\image\p1"
save_root2 = r"D:\고추ood\image\p2"
 
for path in pepper1:
    data = json.load(open(path, "r"))
    shutil.copyfile(path, os.path.join(label_dir1, path.split("\\")[-1]))
    shutil.copyfile(os.path.join(img_root, data['description']['image']), os.path.join(save_root1, data['description']['image']))
    
for path in pepper2:
    data = json.load(open(path, "r"))
    shutil.copyfile(path, os.path.join(label_dir2, path.split("\\")[-1]))
    shutil.copyfile(os.path.join(img_root, data['description']['image']), os.path.join(save_root2, data['description']['image']))

In [ ]:
# meta data 만들기

In [ ]:
root = r"D:\노지 작물 질병 진단 이미지"
train_dir = os.path.join(root, "train")
label_dir = os.path.join(train_dir, "label")

label_files_0 = glob(os.path.join(label_dir, "*_0"))
label_files_1 = glob(os.path.join(label_dir, "*_1"))
label_files_2 = glob(os.path.join(label_dir, "*_2"))

In [ ]:
code, date, h, w, task, ty, region, disease, crop_list, area, grow, risk, bbox = [], [], [], [], [], [],[], [], [], [], [], [], []
img_path = []

for crops in label_files_0:
    crop_files = glob(crops + "\*")
    for crop in crop_files:
        data = json.load(open(crop, "r"))
        code.append(0)
        
        if disease_aware_decoder[str(data['annotations']['disease']).zfill(2)] == 0:
            disease.append(crop_aware_decoder_kr[str(data['annotations']['crop']).zfill(2)])
        else:
            disease.append(disease_aware_decoder[str(data['annotations']['disease']).zfill(2)])
#         disease.append(data['annotations']['disease'])
        date.append(data['description']['date'])
        h.append(data['description']['height'])
        w.append(data['description']['width'])
        task.append(data['description']['task'])
        region.append(data['description']['region'])
        
        crop_list.append(data['annotations']['crop'])
        area.append(data['annotations']['area'])
        grow.append(data['annotations']['grow'])
        risk.append(data['annotations']['risk'])
        bbox.append(data['annotations']['points'])
           
        crop_dir = os.path.join(train_dir + '/images', crop.split("\\")[-2])
        image_path = os.path.join(crop_dir, data['description']['image'])
        img_path.append(image_path)

In [ ]:
df0 = pd.DataFrame({"code":code,
                   "disease":disease,
                   "date":date,
                   "height":h,
                   "width":w,
                   "task":task,
                   "region":region,
                   "crop":crop_list,
                   "area":area,
                   "risk":risk,
                   "bbox":bbox,
                    "path":img_path
                   })

In [ ]:
code, date, h, w, task, ty, region, disease, crop_list, area, grow, risk, bbox = [], [], [], [], [], [],[], [], [], [], [], [], []
img_path = []

for crops in label_files_1:
    crop_files = glob(crops + "\*")
    for crop in crop_files:
        data = json.load(open(crop, "r"))
        code.append(1)
        if disease_aware_decoder[str(data['annotations']['disease']).zfill(2)] == 0:
            disease.append(crop_aware_decoder_kr[str(data['annotations']['crop']).zfill(2)])
        else:
            disease.append(disease_aware_decoder[str(data['annotations']['disease']).zfill(2)])
        date.append(data['description']['date'])
        h.append(data['description']['height'])
        w.append(data['description']['width'])
        task.append(data['description']['task'])
        region.append(data['description']['region'])
        
        crop_list.append(data['annotations']['crop'])
        area.append(data['annotations']['area'])
        grow.append(data['annotations']['grow'])
        risk.append(data['annotations']['risk'])
        bbox.append(data['annotations']['points'])
           
        crop_dir = os.path.join(train_dir + '/images', crop.split("\\")[-2])
        image_path = os.path.join(crop_dir, data['description']['image'])
        img_path.append(image_path)

In [ ]:
df1 = pd.DataFrame({"code":code,
                   "disease":disease,
                   "date":date,
                   "height":h,
                   "width":w,
                   "task":task,
                   "region":region,
                   "crop":crop_list,
                   "area":area,
                   "risk":risk,
                   "bbox":bbox,
                    "path":img_path
                   })

In [ ]:
code, date, h, w, task, ty, region, disease, crop_list, area, grow, risk, bbox = [], [], [], [], [], [],[], [], [], [], [], [], []
img_path = []

for crops in label_files_2:
    crop_files = glob(crops + "\*")
    for crop in crop_files:
        data = json.load(open(crop, "r"))
        code.append(2)
        if disease_aware_decoder[str(data['annotations']['disease']).zfill(2)] == 0:
            disease.append(crop_aware_decoder_kr[str(data['annotations']['crop']).zfill(2)])
        else:
            disease.append(disease_aware_decoder[str(data['annotations']['disease']).zfill(2)])
        date.append(data['description']['date'])
        h.append(data['description']['height'])
        w.append(data['description']['width'])
        task.append(data['description']['task'])
        region.append(data['description']['region'])
        
        crop_list.append(data['annotations']['crop'])
        area.append(data['annotations']['area'])
        grow.append(data['annotations']['grow'])
        risk.append(data['annotations']['risk'])
        bbox.append(data['annotations']['points'])
           
        crop_dir = os.path.join(train_dir + '/images', crop.split("\\")[-2])
        image_path = os.path.join(crop_dir, data['description']['image'])
        img_path.append(image_path)

In [ ]:
df2 = pd.DataFrame({"code":code,
                   "disease":disease,
                   "date":date,
                   "height":h,
                   "width":w,
                   "task":task,
                   "region":region,
                   "crop":crop_list,
                   "area":area,
                   "risk":risk,
                   "bbox":bbox,
                    "path":img_path
                   })

In [ ]:
# test data meta 만들기

root = r"D:\노지 작물 질병 진단 이미지"
valid_dir = os.path.join(root, "valid")
label_dir = os.path.join(valid_dir, "label")

label_files_0 = glob(os.path.join(label_dir, "*_0"))
label_files_1 = glob(os.path.join(label_dir, "*_1"))
label_files_2 = glob(os.path.join(label_dir, "*_2"))

In [ ]:
code, date, h, w, task, ty, region, disease, crop_list, area, grow, risk, bbox = [], [], [], [], [], [],[], [], [], [], [], [], []
img_path = []

for crops in label_files_0:
    crop_files = glob(crops + "\*")
    for crop in crop_files:
        data = json.load(open(crop, "r"))
        code.append(0)
        
        if disease_aware_decoder[str(data['annotations']['disease']).zfill(2)] == 0:
            disease.append(crop_aware_decoder_kr[str(data['annotations']['crop']).zfill(2)])
        else:
            disease.append(disease_aware_decoder[str(data['annotations']['disease']).zfill(2)])
#         disease.append(data['annotations']['disease'])
        date.append(data['description']['date'])
        h.append(data['description']['height'])
        w.append(data['description']['width'])
        task.append(data['description']['task'])
        region.append(data['description']['region'])
        
        crop_list.append(data['annotations']['crop'])
        area.append(data['annotations']['area'])
        grow.append(data['annotations']['grow'])
        risk.append(data['annotations']['risk'])
        bbox.append(data['annotations']['points'])
           
        crop_dir = os.path.join(valid_dir + '/images', crop.split("\\")[-2])
        image_path = os.path.join(crop_dir, data['description']['image'])
        img_path.append(image_path)
        
df0 = pd.DataFrame({"code":code,
                   "disease":disease,
                   "date":date,
                   "height":h,
                   "width":w,
                   "task":task,
                   "region":region,
                   "crop":crop_list,
                   "area":area,
                   "risk":risk,
                   "bbox":bbox,
                    "path":img_path
                   })

In [ ]:
code, date, h, w, task, ty, region, disease, crop_list, area, grow, risk, bbox = [], [], [], [], [], [],[], [], [], [], [], [], []
img_path = []

for crops in label_files_1:
    crop_files = glob(crops + "\*")
    for crop in crop_files:
        data = json.load(open(crop, "r"))
        code.append(1)
        if disease_aware_decoder[str(data['annotations']['disease']).zfill(2)] == 0:
            disease.append(crop_aware_decoder_kr[str(data['annotations']['crop']).zfill(2)])
        else:
            disease.append(disease_aware_decoder[str(data['annotations']['disease']).zfill(2)])
        date.append(data['description']['date'])
        h.append(data['description']['height'])
        w.append(data['description']['width'])
        task.append(data['description']['task'])
        region.append(data['description']['region'])
        
        crop_list.append(data['annotations']['crop'])
        area.append(data['annotations']['area'])
        grow.append(data['annotations']['grow'])
        risk.append(data['annotations']['risk'])
        bbox.append(data['annotations']['points'])
           
        crop_dir = os.path.join(valid_dir + '/images', crop.split("\\")[-2])
        image_path = os.path.join(crop_dir, data['description']['image'])
        img_path.append(image_path)

df1 = pd.DataFrame({"code":code,
                   "disease":disease,
                   "date":date,
                   "height":h,
                   "width":w,
                   "task":task,
                   "region":region,
                   "crop":crop_list,
                   "area":area,
                   "risk":risk,
                   "bbox":bbox,
                    "path":img_path
                   })

In [ ]:
code, date, h, w, task, ty, region, disease, crop_list, area, grow, risk, bbox = [], [], [], [], [], [],[], [], [], [], [], [], []
img_path = []

for crops in label_files_2:
    crop_files = glob(crops + "\*")
    for crop in crop_files:
        data = json.load(open(crop, "r"))
        code.append(2)
        if disease_aware_decoder[str(data['annotations']['disease']).zfill(2)] == 0:
            disease.append(crop_aware_decoder_kr[str(data['annotations']['crop']).zfill(2)])
        else:
            disease.append(disease_aware_decoder[str(data['annotations']['disease']).zfill(2)])
        date.append(data['description']['date'])
        h.append(data['description']['height'])
        w.append(data['description']['width'])
        task.append(data['description']['task'])
        region.append(data['description']['region'])
        
        crop_list.append(data['annotations']['crop'])
        area.append(data['annotations']['area'])
        grow.append(data['annotations']['grow'])
        risk.append(data['annotations']['risk'])
        bbox.append(data['annotations']['points'])
           
        crop_dir = os.path.join(valid_dir + '/images', crop.split("\\")[-2])
        image_path = os.path.join(crop_dir, data['description']['image'])
        img_path.append(image_path)
        
df2 = pd.DataFrame({"code":code,
                   "disease":disease,
                   "date":date,
                   "height":h,
                   "width":w,
                   "task":task,
                   "region":region,
                   "crop":crop_list,
                   "area":area,
                   "risk":risk,
                   "bbox":bbox,
                    "path":img_path
                   })